<a href="https://colab.research.google.com/github/JuyeongKime2/Visualization-of-topics-by-MBTI/blob/main/MBTI_Visualization_%EC%A7%84%EC%A7%9C%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Read_csv**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
import io
from google.colab import files

uploaded = files.upload()

# 업로드된 파일의 파일 이름과 내용을 가져오기
filename = list(uploaded.keys())[0]
file_contents = uploaded[filename]

# 파일 내용을 UTF-8로 디코딩
decoded_content = file_contents.decode('utf-8')

# StringIO를 사용하여 파일과 유사한 객체 생성
file_like_object = io.StringIO(decoded_content)

# CSV 파일을 Pandas DataFrame으로 읽기
df = pd.read_csv(file_like_object)

#데이터 확인
df.head()

## **CountVectorizer 객체 생성 및 토픽 중요도 계산**
각 문서에 대해 토픽 분포를 계산

In [ ]:
# CountVectorizer 객체 생성
vectorizer = CountVectorizer()

# 텍스트 벡터화 - 필요한 경우 열 이름 조정
# 'Text'가 올바른 열이 아닌 경우 실제 이름으로 바꿉니다.
# Check the column names in your DataFrame
print(df.columns)

# 'actual_column_name'을 텍스트 데이터가 포함된 올바른 열 이름으로 변경
# 제공된 전역 변수를 기반으로 'posts'가 적합한 후보로 보임
X = vectorizer.fit_transform(df['posts'])

# LDA 모델 생성 및 적합
num_topics = 10  # 적절한 토픽 수 선택
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# 각 문서에 대한 토픽 분포 계산
topic_distributions = lda.transform(X)

# 'type'으로 MBTI 열 이름 설정. DataFrame의 실제 열 이름으로 변경해야 할 수 있습니다.
mbti_column_name = 'type'

# 결과 출력
for i, topic_dist in enumerate(topic_distributions):
    print(f"Document {i} (MBTI: {df[mbti_column_name][i]}): {topic_dist}")

# **각 토픽별로 확률이 높은 단어들을 문자열로 출력**

In [ ]:
import numpy as np

# TF-IDF 벡터화 또는 CountVectorizer를 사용한 경우
feature_names = vectorizer.get_feature_names_out() # Use get_feature_names_out()

# 각 토픽별로 확률이 높은 단어들을 문자열로 출력
top_n_words = 10  # 각 토픽에서 상위 n개의 단어를 출력

for topic_idx, topic in enumerate(lda.components_):
    top_words_indices = topic.argsort()[-top_n_words:][::-1]
    top_words = [feature_names[i] for i in top_words_indices]
    top_words_str = ", ".join(top_words)
    print(f"토픽 {topic_idx+1}: {top_words_str}")

# **히트맵 생성 :집계된 토픽 중요도를 히트맵으로 시각화**

In [ ]:
pip install koreanize-matplotlib

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

# 데이터프레임 생성
df_heatmap = pd.DataFrame(topic_importance, index=[f'Topic {i}' for i in range(lda.n_components)])

# 히트맵 생성
plt.figure(figsize=(13, 13)) #크기 설정
sns.heatmap(df_heatmap, annot=True, cmap='YlGnBu', cbar=True)
plt.title('MBTI 별 토픽 중요도 히트맵')
plt.xlabel('MBTI 유형')
plt.ylabel('토픽')
plt.show()

# **워드클라우드 생성**

상위 10위의 MBTI Topic이 담긴 텍스트 파일을 변수에 할당해서 워드클라우드 출력

In [ ]:
!apt-get -y install fonts-nanum
!fc-cache -fv
from wordcloud import WordCloud

uploaded = files.upload()

# 업로드된 파일의 파일 이름과 내용을 가져오기
filename = list(uploaded.keys())[0]
file_contents = uploaded[filename]

# 파일 내용을 문자열로 디코딩
text = file_contents.decode('utf-8') # Decode the bytes to string

# 중복 단어 제거
words = text.split()
unique_words = set(words)
unique_text = " ".join(unique_words)

# 워드클라우드 생성
wc = WordCloud(width=800,            # 워드클라우드 이미지의 너비 설정
               height=400,           # 워드클라우드 이미지의 높이 설정
               background_color='white', # 워드클라우드 배경색 설정
               colormap='tab20b',     # 워드클라우드 색상 맵 설정
               font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # 워드클라우드에 사용할 폰트 파일 경로 설정
              ).generate(unique_text) # unique_text를 이용하여 워드클라우드 생성

# 워드클라우드 출력
plt.figure(figsize=(10, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis('off')
plt.title("텍스트 파일 워드클라우드 (중복 제거)")
plt.show()